In [1]:
import xgboost as xgb
import pandas  as pd
import numpy   as np

In [ ]:
data_import =function(dataname){
  filename = paste('https://raw.githubusercontent.com/avinashbarnwal/GSOC-2019/master/AFT/test/data/neuroblastoma-data-master/data/',dataname,'/',sep="")
  inputFileName = paste(filename,'inputs.csv',sep="")
  labelFileName = paste(filename,'outputs.csv',sep="")
  foldsFileName = paste(filename,'cv/equal_labels/folds.csv',sep="")
  inputs        = read.table(inputFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  labels        = read.table(labelFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  folds         = read.table(foldsFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  res           = list()
  res$inputs    = inputs
  res$labels    = labels
  res$folds     = folds
  return(res)
}

In [ ]:
def data_import(dataName):
    filename = 'https://raw.githubusercontent.com/avinashbarnwal/GSOC-2019/master/AFT/test/data/neuroblastoma-data-master/data/'+dataname+'/'
    inputFileName = filename+'inputs.csv'
    labelFileName = filename+'outputs.csv'
    foldsFileName = filename+'cv/equal_labels/folds.csv'
    inputs        = pd.read_csv(inputFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
    labels        = read.table(labelFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
    folds         = read.table(foldsFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
    res           = list()
    res$inputs    = inputs
    res$labels    = labels
    res$folds     = folds
    return(res)

In [2]:
dataname = 'ATAC_JV_adipose'

In [3]:
filename = 'https://raw.githubusercontent.com/avinashbarnwal/GSOC-2019/master/AFT/test/data/neuroblastoma-data-master/data/'+dataname+'/'
inputFileName = filename+'inputs.csv'
labelFileName = filename+'outputs.csv'
foldsFileName = filename+'cv/equal_labels/folds.csv'
inputs        = pd.read_csv(inputFileName,index_col='sequenceID')
labels        = pd.read_csv(labelFileName,index_col='sequenceID')
folds         = pd.read_csv(foldsFileName,index_col='sequenceID')

In [4]:
inputs.replace([-float('inf'),float('inf')],np.nan,inplace=True)

In [5]:
missingCols = inputs.isnull().sum()

In [6]:
missingCols = list(missingCols[missingCols>0].index)

In [7]:
inputs.drop(missingCols,axis=1,inplace=True)

In [8]:
varCols     = inputs.apply(lambda x: np.var(x))
zeroVarCols = list(varCols[varCols==0].index)

In [9]:
inputs.drop(zeroVarCols,axis=1,inplace=True)

In [10]:
labels['min.log.lambda'] = labels['min.log.lambda'].apply(lambda x: np.exp(x))
labels['max.log.lambda'] = labels['max.log.lambda'].apply(lambda x: np.exp(x))

In [11]:
folds[folds['fold']==1]
test_id       = list(folds[folds['fold']==1].index)
train_id      = list(folds[folds['fold']!=1].index)
X             = inputs[inputs.index.isin(train_id)]
X_val         = inputs[inputs.index.isin(test_id)]
y_label       = labels[labels.index.isin(train_id)]
y_label_test  = labels[labels.index.isin(test_id)]
y_lower       = y_label['min.log.lambda']
y_upper       = y_label['max.log.lambda']
y_lower_val   = y_label_test['min.log.lambda']
y_upper_val   = y_label_test['max.log.lambda']

In [ ]:
getXY<-function(foldNo,folds,inputs,labels){
    test.id       = rownames(subset(folds,fold==foldNo))
    train.id      = rownames(subset(folds,fold!=foldNo))
    X             = subset(inputs,rownames(inputs) %in% train.id)
    X             = as.matrix(X)
    X.val         = subset(inputs,rownames(inputs) %in% test.id)
    X.val         = as.matrix(X.val)
    y.label       = subset(labels,rownames(labels) %in% train.id)
    y.label.test  = subset(labels,rownames(labels) %in% test.id)
    y.lower       = as.matrix(y.label$min.log.lambda)
    y.upper       = as.matrix(y.label$max.log.lambda)
    y.lower.val   = as.matrix(y.label.test$min.log.lambda)
    y.upper.val   = as.matrix(y.label.test$max.log.lambda)
    res           = list()
    res$X         = X
    res$X.val     = X.val
    res$y.lower      = y.lower
    res$y.lower.val  = y.lower.val
    res$y.upper      = y.upper
    res$y.upper.val  = y.upper.val
    return(res)
}

In [ ]:
trainModel = function(X,X_val,y_lower,y_upper,y_lower_val,y_upper_val,param,num_round){
    dtrain = xgb.DMatrix(X)
    setinfo(dtrain,'label_lower_bound', y_lower)
    setinfo(dtrain,'label_upper_bound', y_upper)
    dtest = xgb.DMatrix(X_val)
    setinfo(dtest,'label_lower_bound', y_lower_val)
    setinfo(dtest,'label_upper_bound', y_upper_val)
    watchlist = list(eval = dtest, train = dtrain)
    bst       = xgb.train(param, dtrain, num_round, watchlist,verbose = 0)
    min_val_error = min(bst$evaluation_log$eval)
    return(min_val_error)
    #return(0)
}

In [14]:
dtrain = xgb.DMatrix(X)
dtrain.set_float_info("label_lower_bound",y_lower)
dtrain.set_float_info("label_upper_bound",y_higher)
    
dtest  = xgboost.DMatrix(X_val)
dtest.set_float_info("label_lower_bound",y_lower_val)
dtest.set_float_info("label_upper_bound",y_higher_val)
    
bst    = xgboost.train(params,dtrain,num_boost_round=100,evals=[(dtrain,"train"),(dtest,"test")],evals_result=res)

In [ ]:
dtrain.setinfo(dtrain,'label_upper_bound', y_upper)
dtest = xgb.DMatrix(X_val)
dtest.setinfo(dtest,'label_lower_bound', y_lower_val)
dtest.setinfo(dtest,'label_upper_bound', y_upper_val)
watchlist = list(eval = dtest, train = dtrain)